### Import necessary libraries 

In [11]:
import cv2
from keras.models import load_model
import numpy as np
import random
import time

### Define class called RPS_game, that will hold important methods that can be called on each iteration of the game

In [12]:
class RPS_game:

    def __init__(self):
        self.model = load_model('keras_model.h5')
        self.cap = cv2.VideoCapture(0)
        self.options = ['Rock', "Paper", "Scissors", "Nothing"]
        self.points_computer = 0
        self.points_user = 0
        self.rounds_played = 0
    
    #display instructions for the computer vision version of RPS
    def game_instructions(self):
        print(
            '''Welcome to an interactive version of Rock Paper Scissors!
When your camera will turn on, you will see the instructions on the screen.
Show your choice after the GO! cue.
ENJOY!
            '''
        )

    #opens user camera and starts acquiring frames
    def start_video(self):
        
        #displays instruction for starting a round & stopping the game
        self.data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
        ret, frame = self.cap.read()
        #crops camera to dimensions similar to ML model training
        cropped_frame = frame[200:1000, 300:1100]
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(frame, 'Hold s to start countdown', (0, 100), font, 2, (118, 237, 250), 2, cv2.LINE_AA)
        cv2.putText(frame, 'Hold q to stop game', (800, 700), font, 2, (78, 99, 235), 2, cv2.LINE_AA)
        cv2.imshow('frame', frame)

    #stops camera and closes windows
    def stop_video(self):
        self.cap.release()
        # Destroy all the windows
        if self.rounds_played == 5:
            print(f'You played 5 rounds, the game is finished.')
        else:
            print(f'You left game before the outcome was determined')
        cv2.waitKey(1)
        cv2.destroyAllWindows()
        cv2.waitKey(1)

    # gets predicitons from model
    def get_prediction(self):
        predictions = self.model.predict(self.data)
        idx = np.argmax(predictions[0])
        return idx
        
    # gets computer and user choices 
    def get_computer_and_user_choices(self):

        self.computer_choice = random.choice(self.options[:3])
        idx = self.get_prediction()

        if idx == 3:
            #print ("No category detected - try again!")
            self.user_choice = 'None'
        else:
            self.user_choice = self.options[idx]
            print(f'You chose {self.user_choice}')
            print(f'The computer chose {self.computer_choice}')

    # gets winner between user and computer & counts number of rounds played
    def get_winner(self):
         
        if self.computer_choice == 'Rock' and self.user_choice == 'Scissors':
            self.winner = 'the computer'
            self.points_computer += 1
            self.rounds_played += 1

        elif self.computer_choice == 'Rock' and self.user_choice == 'Paper':
            self.winner = 'you'
            self.points_user += 1
            self.rounds_played += 1

        elif self.computer_choice == 'Scissors' and self.user_choice == 'Paper':
            self.winner = 'the computer'
            self.points_computer += 1
            self.rounds_played += 1


        elif self.computer_choice == 'Scissors' and self.user_choice == 'Rock':
            self.winner = 'you'
            self.points_user += 1
            self.rounds_played += 1

        elif self.computer_choice == 'Paper' and self.user_choice == 'Rock':
            self.winner = 'the computer'
            self.points_computer += 1
            self.rounds_played += 1

        elif self.computer_choice == 'Paper' and self.user_choice == 'Scissors':
            self.winner = 'you'
            self.points_user += 1
            self.rounds_played += 1

        elif self.computer_choice == self.user_choice:
            self.winner = 'Nobody won this round!' 
            self.rounds_played += 1

        else:
            print('Camera did not detect user choice - repeat round')
        
        if self.winner == 'you' or self.winner == 'the computer':
            print(f'Round {self.rounds_played}: The winner of this round is {self.winner}\n') 
        elif self.winner == 'Nobody won this round!':
            print(f'Round {self.rounds_played}: {self.winner}\n')
 


### Define function to play game

In [13]:
def play_game():
     
    game = RPS_game()
    #displays instructions for playing RPS
    game.game_instructions()

    while True:
        #initialises timer for countdown
        timer = 3
        #opens camera
        game.start_video()

        #if user press s countdown starts
        if cv2.waitKey(80) & 0xFF == ord('s'):

            #returns time at the start of countdown
            prev = time.time()

            # if timer > 0 -timer is displayed on screen 
            while timer > 0:
                font = cv2.FONT_HERSHEY_TRIPLEX
                ret, frame = game.cap.read()
                cropped_frame = frame[200 : 1000, 300 : 1100]
                cv2.putText(frame, str(timer), (200, 250), font, 5, (255, 145, 0), 4, cv2.LINE_AA)
                cv2.imshow('frame', frame)
                cv2.waitKey(1)
                cur = time.time()
                
                # decrease timer by 1 second
                if cur - prev >= 1:
                    prev = cur
                    timer = timer - 1

            # when timer reaches 0, display GO! cue on the screen
            else:
                ret, frame = game.cap.read()
                cropped_frame = frame[200 : 1000, 300 : 1100]
                font = cv2.FONT_HERSHEY_TRIPLEX
                cv2.putText(frame, 'GO!', (200, 250), font, 5, (255, 145, 0), 4, cv2.LINE_AA)
                cv2.imshow('frame', frame)
                cv2.waitKey(30)

                resized_frame = cv2.resize(frame, (224, 224), interpolation = cv2.INTER_AREA)
                image_np = np.array(resized_frame)
                normalized_image = (image_np.astype(np.float32) / 127.0) - 1 # Normalize the image
                game.data[0] = normalized_image
                
                #calls class methods to extract desired information about game outcome
                game.get_computer_and_user_choices()
                game.get_winner()

                #after playing 5 rounds outcome is displayed
                if game.rounds_played == 5:

                    if game.points_computer > game.points_user :
                        print("You lost, the computer won :(")
                    elif game.points_user > game.points_computer:
                        print("You won! Go celebrate")
                    
                    game.stop_video()
                    break
            
        #if user press q camera and game stops
        elif cv2.waitKey(80) & 0xFF == ord('q'):
            
            game.stop_video()
            break    
      
        #reinitialise timer back to 3s - to prepare for another round
        else:
            timer = int(3)
        
    

## PLAY AHEAD!!

In [15]:
play_game()

Welcome to an interactive version of Rock Paper Scissors!
When your camera will turn on, you will see the instructions on the screen.
Show your choice after the GO! cue.
ENJOY!
            
You chose Rock
The computer chose Paper
Round 1: The winner of this round is the computer

You chose Scissors
The computer chose Paper
Round 2: The winner of this round is you

You chose Paper
The computer chose Paper
Round 3: Nobody won this round!

Camera did not detect user choice - repeat round
Round 3: Nobody won this round!

You chose Paper
The computer chose Rock
Round 4: The winner of this round is you

You chose Paper
The computer chose Paper
Round 5: Nobody won this round!

You won! Go celebrate
You played 5 rounds, the game is finished.
